In [ ]:
# !pip install nltk
# import nltk
# nltk.download('punkt')
nltk.download('stopwords')

In [1]:
import pandas as pd
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

# Data

In [2]:
data = pd.read_csv("data/ACL_data.csv").drop(columns = 'Unnamed: 0' )
data.head()

,YEAR,TITLE,ABSTRACT,PUBLISHER,BOOKTITLE,CATEGORY
0,2022,A Systematic Survey of Text Worlds as Embodied...,Text Worlds are virtual environments for embod...,Association for Computational Linguistics,Proceedings of the 3rd Wordplay: When Language...,INPROCEEDINGS
1,2022,A Minimal Computational Improviser Based on Or...,A prototype system for playing a minimal impro...,Association for Computational Linguistics,Proceedings of the 3rd Wordplay: When Language...,INPROCEEDINGS
2,2022,Craft an Iron Sword: Dynamically Generating In...,Non-Player Characters (NPCs) significantly enh...,Association for Computational Linguistics,Proceedings of the 3rd Wordplay: When Language...,INPROCEEDINGS
3,2022,A Sequence Modelling Approach to Question Answ...,Interactive Question Answering (IQA) requires ...,Association for Computational Linguistics,Proceedings of the 3rd Wordplay: When Language...,INPROCEEDINGS
4,2022,Automatic Exploration of Textual Environments ...,The purpose of this extended abstract is to di...,Association for Computational Linguistics,Proceedings of the 3rd Wordplay: When Language...,INPROCEEDINGS


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32352 entries, 0 to 32351
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   YEAR       32352 non-null  int64 
 1   TITLE      32352 non-null  object
 2   ABSTRACT   32246 non-null  object
 3   PUBLISHER  32352 non-null  object
 4   BOOKTITLE  32352 non-null  object
 5   CATEGORY   32352 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


In [4]:
data.YEAR.value_counts()

2020    6574
2021    5774
2019    4315
2018    3363
2022    3054
2017    2495
2016    1778
2010     832
2014     819
2012     762
2008     735
2006     622
2004     201
2011     117
2015     107
2009     104
2002      88
2007      88
2005      88
2000      79
2003      64
2001      61
1998      51
1989      47
1993      43
1995      33
1997      31
1991      27
Name: YEAR, dtype: int64

In [12]:
data.PUBLISHER.value_counts()

Association for Computational Linguistics                                                 21004
European Language Resources Association (ELRA)                                             4147
ATALA                                                                                      1375
European Language Resources Association                                                    1104
The COLING 2016 Organizing Committee                                                        739
International Committee on Computational Linguistics                                        654
INCOMA Ltd.                                                                                 573
Association for Machine Translation in the Americas                                         384
International Committee for Computational Linguistics                                       307
Asian Federation of Natural Language Processing                                             253
Chinese Information Processing Society o

In [13]:
data.CATEGORY.value_counts()

INPROCEEDINGS    32352
Name: CATEGORY, dtype: int64

## Cleaning and tokenize

In [99]:
# En este caso particular no hace falta limpiar los textos
def clean_text(text):
    if isinstance(text, pd.Series):
        return text.str.strip().values
    elif isinstance(text, str):
        return text.strip()

def tokenizer(text):
    #return [w for w in word_tokenize(text) if w.isalpha()] # si solo nos interesan palabras
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return text.apply(word_tokenize)



In [108]:
# word_tokenize(clean_text(data.ABSTRACT))
tokenizer(clean_text(data.ABSTRACT))


AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [ ]:
data.ABSTRACT.apply(word_tokenize)

In [ ]:
x = data.ABSTRACT.apply(word_tokenize)

In [ ]:
tokenizer(clean_text(data.ABSTRACT))#[:10]

La funcion anterior rompe con series y dataframes

In [92]:
print( type( clean_text( data.ABSTRACT[0]) ))
print( type(clean_text( data.ABSTRACT[:1]) ) )

# tokenizer(clean_text(data.ABSTRACT[:1] ))#[:10]

<class 'str'>
<class 'numpy.ndarray'>


In [ ]:
data.ABSTRACT[:1].values
# data.iloc[:1]

## Stoplist

In [ ]:
stoplist = stopwords.words("english")
stoplist[:15]

In [ ]:
stoplist_tokenized = []
for w in stoplist:
    stoplist_tokenized = stoplist_tokenized + tokenizer(w)
stoplist_tokenized[:15]